In [1]:
# !pip install sqlalchemy
#!conda install -y -q pymssql
# !pip install Pymssql

In [2]:
import sqlalchemy as sa
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData

In [3]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Oratile'

In [4]:
connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

In [5]:
sql_query = "Select * from claims_data"

In [6]:
df = pd.read_sql_query(sql_query, connection)

In [7]:
df.head()

,age,sex,bmi,steps,children,smoker,region,insurance_claim,claim_amount
0,19,female,27.9,3009,0,yes,southwest,yes,16884.924
1,18,male,33.77,3008,1,no,southeast,yes,1725.5523
2,28,male,33,3009,3,no,southeast,no,0
3,33,male,22.705,10009,0,no,northwest,no,0
4,32,male,28.88,8010,0,no,northwest,yes,3866.8552


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [9]:
## Check the data types of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 9 columns):
age                1338 non-null int64
sex                1338 non-null object
bmi                1338 non-null object
steps              1338 non-null int64
children           1338 non-null object
smoker             1338 non-null object
region             1338 non-null object
insurance_claim    1338 non-null object
claim_amount       1338 non-null object
dtypes: int64(2), object(7)
memory usage: 94.2+ KB


In [10]:
## Convert Numeric values
#df['row_id'] = pd.to_numeric(df['row_id'])
df['bmi'] = pd.to_numeric(df['bmi'])
df['children'] = pd.to_numeric(df['children'])

In [11]:
# Lables
y = df['insurance_claim']

# Features
X = df.drop(['insurance_claim', 'claim_amount'], axis=1)

In [12]:
# Transforming the Features
X_transformed = pd.get_dummies(X, sparse=False, drop_first=True)
X_transformed.head()

,age,bmi,steps,children,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,3009,0,0,1,0,0,1
1,18,33.770,3008,1,1,0,0,1,0
2,28,33.000,3009,3,1,0,0,1,0
3,33,22.705,10009,0,1,0,1,0,0
4,32,28.880,8010,0,1,0,1,0,0


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.20, random_state = 47)

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [16]:
tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=4, random_state=47)

In [17]:
# train initial model
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=47, splitter='best')

In [18]:
# Making the predictions
pred1 = tree.predict(X_test)

In [19]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [20]:
# performance matrics and model eveluation using sklearn.metrics inbuilt classification metric
print(classification_report(y_test, pred1))

              precision    recall  f1-score   support

          no       0.87      0.92      0.90       102
         yes       0.95      0.92      0.93       166

    accuracy                           0.92       268
   macro avg       0.91      0.92      0.91       268
weighted avg       0.92      0.92      0.92       268



In [21]:
sql_query_test = "Select * from test_set"
df_test = pd.read_sql_query(sql_query_test, connection)
df_test.head()

,row_id,age,sex,bmi,steps,children,smoker,region
0,1,27,male,28.88,8049,0,no,southwest
1,2,20,female,26.265,7970,3,yes,southwest
2,3,32,female,26.025,2973,0,no,southwest
3,4,19,female,22.085,10427,1,yes,southwest
4,5,20,female,28.685,3972,0,yes,southwest


In [22]:
### Check data types
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
row_id      300 non-null object
age         300 non-null int64
sex         300 non-null object
bmi         300 non-null object
steps       300 non-null int64
children    300 non-null object
smoker      300 non-null object
region      300 non-null object
dtypes: int64(2), object(6)
memory usage: 18.9+ KB


In [23]:
## Convert Numeric values
df_test['row_id'] = pd.to_numeric(df_test['row_id'])
df_test['bmi'] = pd.to_numeric(df_test['bmi'])
df_test['children'] = pd.to_numeric(df_test['children'])

In [24]:
X_submission = df_test.drop('row_id', axis=1)

In [25]:
X_submission_transformed = pd.get_dummies(X_submission, sparse=False, drop_first=True)
X_submission_transformed.head()

,age,bmi,steps,children,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,27,28.880,8049,0,1,0,0,0,1
1,20,26.265,7970,3,0,1,0,0,1
2,32,26.025,2973,0,0,0,0,0,1
3,19,22.085,10427,1,0,1,0,0,1
4,20,28.685,3972,0,0,1,0,0,1


In [26]:
## Use trained model on unseen data
predictions = tree.predict(X_submission_transformed)

In [27]:
submission = pd.DataFrame({'row_id' : df_test.row_id,'predictions' : predictions.ravel()}) 

In [28]:
submission.head()

,row_id,predictions
0,1,yes
1,2,no
2,3,yes
3,4,yes
4,5,yes


In [49]:
submission = pd.DataFrame({'row_id' : df_test.row_id,'predictions' : predictions.ravel()}) 

In [50]:
#Save the DataFrame to SQL
submission.to_sql('classification_model_results', con=connection, if_exists='append', index=False,method='multi',chunksize=500)

In [51]:
connection.table_names()

['claims_data',
 'classification_model_results',
 'english_players',
 'football_players',
 'test_set']